In [9]:
import json
import os
from dotenv import load_dotenv
from openai import OpenAI

# 환경 변수 파일 경로 설정
PATH = 'apikey.env'

def load_env_file(env_path):
    load_dotenv(env_path)

def read_faq_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        faq_dict = json.load(file)
    return faq_dict

def chatgpt(user_input, faq_dict, model="gpt-3.5-turbo"):
    """
    FAQ 데이터를 기반으로 GPT 모델을 통해 응답을 생성합니다.
    
    Args:
        user_input (str): 사용자 입력 데이터 문자열
        faq_dict (dict): 질문과 답변이 저장된 딕셔너리
        model (str): 사용할 GPT 모델 (기본값: "gpt-3.5-turbo")

    Returns:
        str: 답변 메시지
    """
    
    # .env 파일에서 환경 변수 로드
    load_env_file(PATH)

    # OpenAI API 키 설정
    api_key_custom = os.getenv('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key_custom)

    # 가장 유사한 질문 찾기 (여기서 랭체인 기능을 사용할 수 있음)
    closest_question = max(faq_dict.keys(), key=lambda q: similar(q, user_input))
    document_content = faq_dict[closest_question]

    # 프롬프트 구성
    messages = [
        {"role": "system", "content": "You are an advisor in a stock auto-trading application."},
        {"role": "user", "content": f"Document content: {document_content}"},
        {"role": "user", "content": f"User's question: {user_input}"}
    ]

    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0
        )
        review = response.choices[0].message.content
        return review
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def similar(a, b):
    from difflib import SequenceMatcher
    return SequenceMatcher(None, a, b).ratio()

if __name__ == "__main__":
    json_path = 'faq_data.json'
    faq_dict = read_faq_json(json_path)
    user_question = '산출물에서 제출 기준이 필수인 항목을 알려줘'
    print(chatgpt(user_question, faq_dict))


산출물에서 제출 기준이 필수인 항목은 다음과 같습니다:
1. 요구사항 정의서
2. 서비스 구성도(시스템 구성도)
3. 서비스 흐름도(데이터 흐름도)
4. 메뉴 구성도
5. 화면 설계서
6. 엔티티 관계도
7. 기능 처리도(기능 흐름도)
8. 알고리즘 명세서/설명서
9. 데이터 수집처리 정의서
10. 프로그램 목록
11. 테이블 정의서
12. 핵심 소스코드
